In [146]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
from random import random
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import matplotlib
import matplotlib.pyplot as plt

In [147]:
# Load data
df_laptops_test = pd.read_csv(r"../../data/test.csv", sep=',')
df_laptops_train = pd.read_csv(r"../../data/train.csv", sep=',')

### Cleaning data and solving missing values

In [148]:
# Get all touchscreen values to lower case letters (f.ex: from 'Glossy' --> 'glossy')
def lower_case_screen(df):
    return df['screen_surface'].replace({'Glossy': 'glossy', 'Matte': 'matte'}, inplace=True)

lower_case_screen(df_laptops_train)
lower_case_screen(df_laptops_test)

In [149]:
# Detect missing values
def detect_and_fill_all_missing_values_with_correct_nan(df):
    df.fillna(value=np.nan,inplace=True)

detect_and_fill_all_missing_values_with_correct_nan(df_laptops_train)
detect_and_fill_all_missing_values_with_correct_nan(df_laptops_test)

In [150]:
# Replace NaN screen_surface values with 'glossy' or 'matte' at random
def replace_nan_screen_surface(df):
    for i in range(0,len(df)):
        if type(df.screen_surface[i]) == float:
            if random() >= 0.5:
                df.at[i,'screen_surface'] = 'glossy'
            else:
                df.at[i,'screen_surface'] = 'matte'

replace_nan_screen_surface(df_laptops_train)
replace_nan_screen_surface(df_laptops_test)

In [151]:
# Replace weight missing values by the median of all the weights
def replace_weight_nans_with_median(df):
    df['weight'] = df['weight'].fillna(value=df['weight'].median())

replace_weight_nans_with_median(df_laptops_train)
replace_weight_nans_with_median(df_laptops_test)

In [152]:
# Replace missing OS and OS_details based on brand
def replace_missing_os_and_os_details(df):
    for i in range(0,len(df)):
        if type(df.os[i]) == float:        # When the value is missing
            if 'apple' in df.brand[i].lower():
                df.at[i,'os_details'] = 'macOS Catalina'
                df.at[i,'os'] = 'macOS'
            else:
                df.at[i,'os_details'] = 'Windows'
                df.at[i,'os'] = 'Windows 10'
                
replace_missing_os_and_os_details(df_laptops_train)
replace_missing_os_and_os_details(df_laptops_test)

In [153]:
def resolve_last_missing_values(df):
    # Make index and column arrays to then use in the creation of the new df
    index_array = np.array(range(0,len(df)))
    column_names_array = np.array(list(df.columns), dtype=object)
    
    # replace rest of missing values with most frequent simple imputer
    imp = SimpleImputer(strategy="most_frequent")
    temp_array = imp.fit_transform(df)

    no_nulls_df = pd.DataFrame(data=temp_array[0:,0:],index=index_array, columns=column_names_array)
    
    return no_nulls_df

In [154]:
no_nulls_train = resolve_last_missing_values(df_laptops_train)
no_nulls_test = resolve_last_missing_values(df_laptops_test)

In [155]:
# Remove unimportant columns
def remove_unimportant_columns(df):
    return df.drop(columns = ['id',
                              'name',         # Name information is to be found in other columns
                              'base_name',    # Base name is partially in brand
                              'os',           # OS_details is more important
                              'discrete_gpu', # Information contained in GPU information
                              'cpu_details']) # Only CPU column is good

train_df = remove_unimportant_columns(no_nulls_train)
test_df = remove_unimportant_columns(no_nulls_test)

In [156]:
# Encode glossy and matte as 1 and 0 respectively
def dummy_encode_screen_surface(df):
    for i in range(0,len(df)):
        if df.screen_surface[i] == 'glossy':
            df.at[i,'screen_surface'] = 1
        else:
            df.at[i,'screen_surface'] = 0

dummy_encode_screen_surface(train_df)
dummy_encode_screen_surface(test_df)

In [157]:
def divide_data_cat_and_num(df):
    if 'min_price' in df.columns:
        df_num = df[['screen_size', 
                     'pixels_x', 
                     'pixels_y', 
                     'screen_surface', 
                     'touchscreen', 
                     'detachable_keyboard', 
                     'ram', 
                     'ssd',
                     'storage',
                     'weight',
                     'min_price',
                     'max_price']].astype('int')
    else:
        df_num = df[['screen_size', 
                     'pixels_x', 
                     'pixels_y', 
                     'screen_surface', 
                     'touchscreen', 
                     'detachable_keyboard', 
                     'ram', 
                     'ssd',
                     'storage',
                     'weight']].astype('int')

    df_cat = df[['brand',
                 'cpu', 
                 'gpu', 
                 'os_details']]
    return df_num, df_cat

train_num, train_cat = divide_data_cat_and_num(train_df)
test_num, test_cat = divide_data_cat_and_num(test_df)

In [158]:
# One-hot encode all categorical variables
def one_hot_encode(df_train_categories, df_test_categories):
    full_enc = OneHotEncoder(handle_unknown='ignore')
    full_enc.fit(df_train_categories.to_numpy())
    
    full_transform_train = full_enc.transform(df_train_categories.to_numpy()).toarray()
    full_transform_test = full_enc.transform(df_test_categories.to_numpy()).toarray()
    
    full_transform_train_df = pd.DataFrame(full_transform_train, columns = full_enc.get_feature_names())
    full_transform_test_df = pd.DataFrame(full_transform_test, columns = full_enc.get_feature_names())
    
    return full_transform_train_df, full_transform_test_df

train_cat_ohe, test_cat_ohe = one_hot_encode(train_cat, test_cat)

In [159]:
# Normalize all numerical variables
def normalize(train_num, test_num):
    
    train_num_inputs = train_num[train_num.columns[:-2]]
    train_targets = train_num[train_num.columns[-2:]]
    test_num_inputs = test_num
    
    # Fit one min_max scaler for numerical inputs and one for targets
    min_max_scaler_inputs = preprocessing.MinMaxScaler()
    min_max_scaler_inputs.fit(train_num_inputs.values)
    
    min_max_scaler_targets = preprocessing.MinMaxScaler()
    
    train_scaled = min_max_scaler_inputs.transform(train_num_inputs.values)
    test_scaled = min_max_scaler_inputs.transform(test_num_inputs.values)
    targets_scaled = min_max_scaler_targets.fit_transform(train_targets.values)
    
    # Make final dataframes
    train_norm = pd.DataFrame(train_scaled, columns = train_num_inputs.columns)
    target_norm = pd.DataFrame(targets_scaled, columns = train_targets.columns)
    test_norm = pd.DataFrame(test_scaled, columns = test_num_inputs.columns)
    
    return train_norm, target_norm, test_norm, min_max_scaler_targets

train_norm, target_norm, test_norm, min_max_scaler_targets = normalize(train_num,test_num)

In [160]:
# Bring normalized and one-hot-encoded data together
final_df_train = pd.concat([train_norm, train_cat_ohe, target_norm], axis=1)
final_df_test = pd.concat([test_norm, test_cat_ohe], axis=1)

In [ ]:
# Write final dfs to csv
final_df_train.to_csv(r'../../data/preprocessed_data_train.csv', index = False)
final_df_test.to_csv(r'../../data/preprocessed_data_test.csv', index = False)